Expression for the mean of fleetwide quota: 
\begin{equation}
\sum_k^{\infty}k (\sum_{i=1}^n e^{\mu_{i,T} + \frac{\sigma_i^2}{2} } )   \Phi \left[ \frac{log Q - \mu_{Z,B}}{\sigma_{Z,B}} \right]  \int_0^Q \Phi \left[ -\frac{log(Q-q)-\mu_B}{\sigma_B} \right] \frac{1}{q \sigma_{Z,B} \sqrt{2 \pi}} exp \left[ - \frac{(log (q)- \mu_{Z,B})^2}{2 \sigma_{Z,B}^2}  \right] dq 
\end{equation} 

\begin{align}
& \text{where } \sigma^2_{Z,B} = log \left[ \frac{(k-1) \sum^n_{i=1}e^{2 \mu_{i,B} + \sigma_{i,B}} (e^{\sigma_{i,B}^2} - 1) }{((k-1)(\sum_{i=1}^n e^{\mu_i +\frac{\sigma_{i,B}^2}{2}}))^2 } + 1 \right] 
& \text{and } \mu_{Z,B} = log[(k-1) \sum_{i=1}^{n} e^{2 \mu_{i,B} + \frac{\sigma_{i,B}^2}{2} } ] - \frac{\sigma_{Z,B}^2}{2}
\end{align}

\begin{align}
& \text{where } \sigma^2_{B} = log \left[ \frac{ \sum^n_{i=1}e^{2 \mu_{i,B} + \sigma_{i,B}} (e^{\sigma_{i,B}^2} - 1) }{((\sum_{i=1}^n e^{\mu_i +\frac{\sigma_{i,B}^2}{2}}))^2 } + 1 \right] 
& \text{and } \mu_{B} = log[ \sum_{i=1}^{n} e^{2 \mu_{i,B} + \frac{\sigma_{i,B}^2}{2} } ] - \frac{\sigma_{B}^2}{2}
\end{align}


Expression for the variance of fleetwide quota: 
\begin{align}
 Var[F_{T}] =  \sum_k^{\infty} k \sum_{i=1}^n e^{2 \mu_{i,T} + \sigma_{i,T}^2} (e^{\sigma_i^2} -1)   \Phi \left[ \frac{log Q - \mu_{Z,B}}{\sigma_{Z,B}} \right]  \int_0^Q \Phi \left[ -\frac{log(Q-q)-\mu_B}{\sigma_B} \right] \frac{1}{q \sigma_{Z,B} \sqrt{2 \pi}} exp \left[ - \frac{(log (q)- \mu_{Z,B})^2}{2 \sigma_{Z,B}^2}  \right] dq 
\end{align} 

Expression for the target mean and variance of individual quotas: 
\begin{align}
E[F_T] = \sum_{k=1}^{\infty} \sum_{i=1}^n k(e^{\mu_{i,T} + \frac{\sigma_i^2}{2}}) \Phi \left[ \frac{log (q_i) - \mu_{k-1,B}}{\sigma_{k-1,B}} \right] \int_0^{q_i} \Phi \left[ - \frac{ log(q_i - q\prime) - \mu_{B, k,i}}{\sigma_{B,k,i}} \right] Pr[f_{B,k,i} = q\prime] dq\prime \\
Var[F_T]=  \sum_{k=1}^{\infty} \sum_{i=1}^n  k e^{2\mu_{i,T} + \sigma_i^2} (e^{\sigma_{T,i}^2} - 1)  \Phi \left[ \frac{log (q_i) - \mu_{k-1,B}}{\sigma_{k-1,B}} \right] \int_0^{q_i} \Phi \left[ - \frac{ log(q_i - q\prime) - \mu_{B, k,i}}{\sigma_{B,k,i}} \right] Pr[f_{B,k,i} = q\prime] dq\prime
\end{align} 

In [25]:
import scipy.integrate
from scipy.stats import norm
import matplotlib as plt 
import numpy as np 
from scipy import log 
import operator

# Initialize parameters 

# parameters for both models 
Q = 40000
n = 15 

# first initialize them to have all the same mean, variance
# Keep in mind this parameterization is for the normal distribution describing log returns. 
# mu = 4, sigma = 1.5
# mean of returns = e^(4+(1.5^2)/2) = 168
# variance of returns: e^()

fisher_mu_T = np.empty(n)
fisher_sigma_T = np.empty(n)
fisher_mu_T.fill(4)
fisher_sigma_T.fill(1.5)

fisher_mu_B = np.empty(n)
fisher_sigma_B = np.empty(n)
fisher_mu_B.fill(4)
fisher_sigma_B.fill(1.5)

# mu_B corresponds to the special case that k = 1
def find_sigmasqd_B(k = 1):
    num = k * np.sum( (np.exp(2*fisher_mu_B+fisher_sigma_B)) * (np.exp(fisher_sigma_B*fisher_sigma_B) - 1) )
    denom = pow(k * np.sum( np.exp(fisher_mu_B+fisher_sigma_B/2) ),2) 
    return log(num / denom + 1)

def find_mu_B(sigmasqd_B, k = 1): 
    return log( np.sum( np.exp(2*fisher_mu_B + fisher_sigma_B*fisher_sigma_B / 2) ) ) - sigmasqd_B / 2 

sigmasqd_B = find_sigmasqd_B()
mu_B = find_mu_B(sigmasqd_B)


In [47]:
# Basically we want to show: 
# fleetwide quota has lower variance than case of individual quotas
# mean catch is higher in individual quota case because of free riders ? 
# Still need to model the impact of free-riding vs not free-riding. Possibly in the structure of the u_i

# fleetwide catch: 
# start from k = 0 
def find_fleet_catch(fisher_mu_B, fisher_sigma_B, mu_B, sigmasqd_B, fisher_mu_T, fisher_sigma_T):
    
    def integrand(q): 
        return norm.cdf(-1 * (log(Q - q) - mu_B)/ sigmasqd_B ) * (1/ (q * np.sqrt(sigmasqd_Z_B)*np.sqrt(2*np.pi))) * np.exp(-1* pow((log(q) - mu_Z_B),2)/(2*sigmasqd_Z_B))
                       
     # keep track of some delta, terminate when some precision is met 
    epsilon = 0.00001 
    k = 2
    prev = [0,0]
    result = [0,0]
    delta = 1000
    while (delta > epsilon) and (k < 1000):
        
        sigmasqd_Z_B = find_sigmasqd_B(k-1)
        mu_Z_B = find_mu_B(sigmasqd_Z_B, k-1)
        
        single_catch = np.sum( np.exp(fisher_mu_T+fisher_sigma_T*fisher_sigma_T/2) )
        single_var = np.sum( np.exp(2*fisher_mu_T + fisher_sigma_T*fisher_sigma_T) * (np.exp(fisher_sigma_T) - 1) ) 
        
        # compute this iteration with k 
        common = norm.cdf( (log(Q) - mu_Z_B)/ np.sqrt(sigmasqd_Z_B) ) * scipy.integrate.quad( integrand , 0, Q)[0]
        this = [k * single_catch * common, k * single_var * common] 
                       
        delta = max( [ abs(x) for x in map(operator.sub, this, prev) ] ) 
        prev = this
        result = map(operator.add, result, this)
        
        k += 1
        if (k % 10 == 0): 
            print str(k) + ", this step: " + str(this) + ", result (mean, variance): " + str(result) + ", delta: " + str(delta)
        
    return result

find_fleet_catch(fisher_mu_B, fisher_sigma_B, mu_B, sigmasqd_B, fisher_mu_T, fisher_sigma_T)

10, this step: [1.0243148622681463e-07, 5.9976715393057317e-05], result (mean, variance): [22.885992508110164, 13400.436815952764], delta: 0.000839698541997


[22.885992515405277, 13400.436820224273]

In [1]:
pwd()


u'C:\\Users\\paint_000\\Anaconda'